In [1]:
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import  MultinomialNB
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from math import cos, asin, sqrt, pi

In [2]:
def csrc(df):
    list2 = []
    for i in df.cuisines:
        list2.append(list(set(i)))
    
    vect = CountVectorizer(analyzer = lambda x:x)
    matrix2=vect.fit_transform(list2).toarray()
    
    cosine_similarities_cui = cosine_similarity(matrix2, matrix2)
    return cosine_similarities_cui,vect    

In [3]:
def csrd(df):
    list1 = []
    for i in df.dish_liked:
        list1.append(list(set(i)))
    vect = CountVectorizer(analyzer = lambda x:x)
    matrix1=vect.fit_transform(list1).toarray()
    
    cosine_similarities_dish = cosine_similarity(matrix1, matrix1)
    return cosine_similarities_dish,vect

In [4]:
def csrr(df):
    
    tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
    tfidf_matrix_reviews = tfidf.fit_transform(df['reviews'])
    
    cosine_similarities_rewiew = cosine_similarity(tfidf_matrix_reviews, tfidf_matrix_reviews)
    return cosine_similarities_rewiew,tfidf

In [5]:
def sentiment_lr_mnb(rating):
    vectorizer = CountVectorizer(token_pattern=r'\b\w+\b',ngram_range=(1,2))
    train_matrix= vectorizer.fit_transform(rating['Lemma'])
    lr = LogisticRegression()
    X_train = train_matrix
    #X_test = test_matrix
    y_train = rating['rate_pred1']
    lr.fit(X_train,y_train)
    clf = MultinomialNB(alpha=1)
    clf.fit(X_train,y_train)
    return lr,clf,vectorizer 


In [6]:
def get_max_length(x):
    review_length = []
    for review in x:
        review_length.append(len(review))
    return int(np.ceil(np.mean(review_length)))
def sentiment_lstm(rating):
    x=rating['Lemma']
    y=rating['rate_pred1']
    token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
    token.fit_on_texts(x)
    x= token.texts_to_sequences(x)

    max_length = get_max_length(x)

    x = pad_sequences(x, maxlen=max_length, padding='post', truncating='post')

    total_words = len(token.word_index) + 1
    EMBED_DIM = 32
    LSTM_OUT = 64

    model = Sequential()
    model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
    model.add(LSTM(LSTM_OUT))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
    )
    model.fit(x, y, batch_size = 128, epochs = 5, callbacks=[checkpoint])
    return model,token,max_length

In [7]:
def recommend(index,cosine_similarities,df):
    indices=pd.Series(df.index)
    # Create a list to put top restaurants
    restaurant_recom = []
    for i in index:
        # Find the index of the hotel entered
        idx = indices[indices == i].index[0]

        # Find the restaurants with a similar cosine-sim value and order them from bigges number
        score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)

        # Extract top 30 restaurant indexes with a similar cosine-sim value
        top30_indexes = list(score_series.iloc[0:30].index)

        # Names of the top 30 restaurants
        for each in top30_indexes:
            restaurant_recom.append(list(df.index)[each])

    df_new = pd.DataFrame(columns=df.columns)

    for item in  restaurant_recom:
        df_new = df_new.append(pd.DataFrame(df[df.columns][df.index == item].sample()))
    return df_new
    

In [8]:

def chooseloc(df,la,lo,cost):
    df['man_dist']=0
    df['loc_dist_km']=0
    lm=[]
    lk=[]
    for i,j in (zip(df.iloc[:,16],df.iloc[:,17])):
        p = pi/180
        a = 0.5 - cos((la-float(i))*p)/2 + cos(la*p) * cos(float(i)*p) * (1-cos((lo-float(j))*p))/2
        d=12742 * asin(sqrt(a))
        m=abs(float(i)-la)+abs(float(j)-lo)
        lm.append(m)
        lk.append(d)
    df["man_dist"]=lm
    df["loc_dist_km"]=lk
    df=df[(df['approx_cost(for two people)']<cost)]
    df=df[df['loc_dist_km']<5]
    df=df[df['final_rate']>3]
    df=df.sort_values(by=["final_rate","loc_dist_km"],ascending=(False,True))
    return df

In [9]:
def choosetime(t,user):
    if(t<dt.time(11,30) and t>=dt.time(5,30)):
        mor=user[(user['Time'].dt.time < dt.time(11,30)) & (user['Time'].dt.time >= dt.time(5,30))]
        return mor
    elif(t>=dt.time(11,30) and t<dt.time(16,0)):
        noon=user[(user['Time'].dt.time >= dt.time(11,30)) & (user['Time'].dt.time < dt.time(16,0))]
        return noon
    else:
        night=user[(user['Time'].dt.time >= dt.time(16,0)) | (user['Time'].dt.time < dt.time(5,30))]
        return night

# call once


In [10]:
rating = pd.read_pickle("Ratings2.pkl")
df=pd.read_pickle("pakka_final.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
cosine_similarities_cui,vectc=csrc(df)
cosine_similarities_dish,vectd=csrd(df)
cosine_similarities_rewiew,vectr=csrr(df)
cosine_similarities_final = cosine_similarities_rewiew + cosine_similarities_cui + cosine_similarities_dish
cosine_similarities_dc=cosine_similarities_cui + cosine_similarities_dish
lr,mnb,sentivectlr = sentiment_lr_mnb(rating)
lstm,lstmtoken,max_length = sentiment_lstm(rating)

C:\Users\Noman\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Epoch 1/5
979/979 [==============================] - 30s 28ms/step - loss: 0.3618 - accuracy: 0.8397

Epoch 00001: accuracy improved from -inf to 0.83975, saving model to models\LSTM.h5
Epoch 2/5
979/979 [==============================] - 29s 29ms/step - loss: 0.2874 - accuracy: 0.8782

Epoch 00002: accuracy improved from 0.83975 to 0.87821, saving model to models\LSTM.h5
Epoch 3/5
979/979 [==============================] - 28s 29ms/step - loss: 0.2530 - accuracy: 0.8928

Epoch 00003: accuracy improved from 0.87821 to 0.89280, saving model to models\LSTM.h5
Epoch 4/5
979/979 [==============================] - 29s 30ms/step - loss: 0.2222 - accuracy: 0.9030

Epoch 00004: accuracy improved from 0.89280 to 0.90298, saving model to models\LSTM.h5
Epoch 5/5
979/979 [==============================] - 52s 53ms/step - loss: 0.1961 - accuracy: 0.9147

Epoch 00005: accuracy improved from 0.90298 to 0.91473, saving model to models\LSTM.h5


##### user

In [12]:
def rec_user(user,t,lat,long,df):
    timing=choosetime(t,user)
    c=0
    while(len(timing)==0 and c<2):
        c+=1
        if(dt.time(5,30)<=t and t<dt.time(11,30)):
            t=dt.time(12,0)
            timing=choosetime(t,user)
        elif(dt.time(16,0)>t and t>=dt.time(11,30)):
            t=dt.time(17,0)
            timing=choosetime(t,user)
        else:
            t=dt.time(12,0)
            timing=choosetime(t,user)
    #------no timing
    if(len(timing)==0):
        final_rec=chooseloc(df,lat,long,costt)
        return final_rec
    else:
        test_matrix = sentivectlr.transform(timing['Review'])
        predlr = lr.predict(test_matrix)
        prednb=mnb.predict(test_matrix)
        x_test=timing['Review']
        x_test = lstmtoken.texts_to_sequences(x_test)
        x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')
        pred = list(lstm.predict(x_test))
        for i in range(len(pred)):
            pred[i]=int(round(pred[i][0],0))
        predlstm=np.array(pred)
        majority=list(predlr+prednb+predlstm)
        l=[]
        n=[]
        for i in range(len(majority)):
            if majority[i] >=2:
                l.append(user['Restaurant_Id'][i])
            else:
                n.append(user['Restaurant_Id'][i])
                continue
        if(len(l)>0):
            rec=recommend(l,cosine_similarities_final,df)
        else:
            rec=recommend(n,cosine_similarity_dc)
        rec= rec.drop_duplicates(['name','address'])
        print(rec)
        costt=max(timing['Price'])+100
        final_rec=chooseloc(rec,lat,long,costt)
        return final_rec,timing

In [13]:
import datetime as dt
user=pd.read_csv("user1_final.csv")
user["Time"]=pd.to_datetime((user['Time']))
t=dt.time(10,30)
lat=12.9292731
long=77.5824229
final_rec,timing = rec_user(user,t,lat,long,df)
final_rec

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

                                                address  \
3     1st Floor, Annakuteera, 3rd Stage, Banashankar...   
162   Sri Balaji Complex, 339/3, Kuvempu Nagar, Next...   
6189  267 Near 18Th Cross Signal, Sampige Road, Mall...   
66    727, 23rd Cross, KR Road, 2nd Stage, Banashank...   
4063                13, Near Christ College, Hosur Road   
...                                                 ...   
7307           Near Phoenix Mall, Whitefield, Bangalore   
2032                      14, St. Marks Road, Bangalore   
1377  74/8, Ground Floor, GMR Complex, Outer Ring Ro...   
6945  Beside Leela Palace, Kodihalli, Old Airport Ro...   
4923         270, 100 Feet Road, Indiranagar, Bangalore   

                       name online_order book_table  rate votes  \
3     Addhuri Udupi Bhojana           No         No   3.7    88   
162             Matru Sagar          Yes         No   3.7    31   
6189     Dhanalakshmi Grand          Yes         No   4.0    71   
66             Upahar S

,address,name,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,approx_cost(for two people),list_review_rate,reviews,no_of_cuisines,dish_liked_org,final_rate,lat,long,man_dist,loc_dist_km
3863,"16th Main Road, BTM, Bangalore",So-Fat's Dosas,No,No,3.8,43,BTM,Quick Bites,[dosa],[south indian],200,"[3.0, 4.0, 4.0, 5.0, 3.0, 4.0, 5.0, 5.0, 4.0, ...",amazing collection dosas wonder see much varie...,1,[dosa],4.6,12.911275849999999,77.60456543431182,0.040140,3.124718
2516,"Near Ulsoor Gate Police Station, Cubbonpet, Ci...",Chikkanna Tiffin Room,No,No,4.5,203,City Market,Quick Bites,"[dosa, dosa, dosa, dosa, idli, potato masala, ...",[south indian],150,"[4.0, 4.0, 5.0, 4.0, 5.0, 5.0, 4.0, 4.0]",awesome khali masala dose also fairly wl known...,1,"[dosa, dosa, dosa, dosa, idli, potato masala, ...",4.5,12.965717999999999,77.5762705372058,0.042597,4.106966
1281,"6, 9th Cross, 9th Main, 2nd Block, Jayanagar, ...",Bengaluru Cafe,No,No,4.4,134,Jayanagar,Quick Bites,"[kesari bath, dosa, kharabath, tea, coffee, id...",[south indian],150,"[3.0, 4.0, 4.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, ...",well cafe eye catching jayanagar food good opt...,1,"[kesari bath, dosa, kharabath, tea, coffee, id...",4.4,12.9292731,77.5824229,0.000000,0.000000
378,"32, Gandhi Bazaar, Near Gandhi Bazaar Circle, ...",Vidyarthi Bhavan,No,No,4.4,4460,Basavanagudi,Quick Bites,"[filter kaapi, coffee, khara bhath, maddur vad...",[south indian],150,"[5.0, 5.0, 4.0, 5.0, 4.0, 5.0, 4.0, 5.0, 4.0, ...",amazing place amazing food ordered famous masa...,1,"[filter kaapi, coffee, khara bhath, maddur vad...",4.4,12.9417261,77.5755021,0.019374,1.574791
1207,"118,13th Main, 29th Cross, 4th Block, Jayanaga...",Davangere Benna Dose Hotel,Yes,No,4.1,90,Jayanagar,Quick Bites,"[dosa, idli, coffee]",[south indian],100,"[5.0, 4.0, 5.0, 4.0, 5.0, 5.0, 4.0]",favorite spot benne dosa loved crispy aromatic...,1,"[dosa, idli, coffee]",4.3,12.9292731,77.5824229,0.000000,0.000000
64,"49/1, Subbaraoshetty Road, Netkallappa Circle,...",Sri Guru Kottureshwara Davangere Benne Dosa,Yes,No,4.1,558,Basavanagudi,Quick Bites,"[dosa, mirchi bajji, coffee, poha, tea, dosa, ...",[south indian],150,"[4.0, 4.0, 4.0, 5.0, 4.0, 3.0, 3.0, 4.0, 5.0, ...",goto benne dosa placethe paddu must trythe chu...,1,"[dosa, mirchi bajji, coffee, poha, tea, dosa, ...",4.0,12.9417261,77.5755021,0.019374,1.574791
1273,"Diagonal Road, Sajjan Rao Circle, VV Puram, Ba...",Hotel Janatha,No,No,4.1,52,Basavanagudi,Quick Bites,"[dosa, dosa, dosa, vada, coffee, dosa, kesari ...",[south indian],150,"[5.0, 4.0, 5.0, 4.0, 4.0, 4.0, 3.0, 4.0, 3.0, ...",love first visit easily visible main road plac...,1,"[dosa, dosa, dosa, vada, coffee, dosa, kesari ...",4.0,12.9417261,77.5755021,0.019374,1.574791
4118,"31/1, 22nd Main, 2nd Phase, JP Nagar, Bangalore",Nammura Upahara,No,No,3.8,40,JP Nagar,Quick Bites,"[dosa, tea, vada, idli, coffee]",[south indian],100,[4.0],place serves good quality south indian food un...,1,"[dosa, tea, vada, idli, coffee]",3.9,12.9072515,77.5782713,0.026173,2.489687
125,"19, V.V.puram National School Road, Basavanagu...",Mulabagilu Dosa,Yes,No,3.6,11,Basavanagudi,Quick Bites,"[dosa, dosa, dosa, dosa, dosa]",[south indian],100,[4.0],mulubagilu dose crispy hot masala dosa famous ...,1,"[dosa, dosa, dosa, dosa, dosa]",3.8,12.9417261,77.5755021,0.019374,1.574791
298,"162 , Kathreguppe Cross Road, 3rd Stage, Banas...",Janahaar,Yes,No,3.8,70,Banashankari,Quick Bites,"[dosa, idli, coffee, dosa]",[south indian],100,"[5.0, 5.0, 5.0, 5.0, 3.0, 3.0, 4.0, 3.0, 3.0, ...",god food one happy customer less menu high tas...,1,"[dosa, idli, coffee, dosa]",3.7,12.965283249999999,77.59445195,0.048039,4.210993


In [ ]:
df['rate'].value_counts()

In [27]:
final_rec

,address,name,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,approx_cost(for two people),list_review_rate,reviews,no_of_cuisines,dish_liked_org,final_rate,lat,long,man_dist,loc_dist_km
1281,"6, 9th Cross, 9th Main, 2nd Block, Jayanagar, ...",Bengaluru Cafe,No,No,4.4,134,Jayanagar,Quick Bites,"[kesari bath, dosa, kharabath, tea, coffee, id...",[south indian],150,"[3.0, 4.0, 4.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, ...",well cafe eye catching jayanagar food good opt...,1,"[kesari bath, dosa, kharabath, tea, coffee, id...",4.4,12.9292731,77.5824229,0.000000,0.000000
1083,"764/44, 37th Cross 18th Main, 4th T Block, Jay...",Mysuru Coffee Thindi,Yes,No,4.1,296,Jayanagar,Quick Bites,"[coffee, dosa, tea, badam milk, vada, idli, dosa]","[south indian, beverages]",200,"[5.0, 3.0, 4.0, 4.0]",one best breakfast n tiffin places idli wadas ...,2,"[coffee, dosa, tea, badam milk, vada, idli, dosa]",4.1,12.9292731,77.5824229,0.000000,0.000000
1119,"36, 12th Main, 27th Cross , 4th Block, Jayanag...",Vakulaa Veg Restaurant,Yes,No,3.9,49,Jayanagar,Quick Bites,"[dosa, coffee, pongal, tea, bonda]","[south indian, north indian]",200,"[4.0, 4.0, 5.0]",best place south indian dishes pocket friendly...,2,"[dosa, coffee, pongal, tea, bonda]",4.1,12.9292731,77.5824229,0.000000,0.000000
64,"49/1, Subbaraoshetty Road, Netkallappa Circle,...",Sri Guru Kottureshwara Davangere Benne Dosa,Yes,No,4.1,558,Basavanagudi,Quick Bites,"[dosa, mirchi bajji, coffee, poha, tea, dosa, ...",[south indian],150,"[4.0, 4.0, 4.0, 5.0, 4.0, 3.0, 3.0, 4.0, 5.0, ...",goto benne dosa placethe paddu must trythe chu...,1,"[dosa, mirchi bajji, coffee, poha, tea, dosa, ...",4.0,12.9417261,77.5755021,0.019374,1.574791
280,"459/30, 30th Cross, 4th Block, Jayanagar, Bang...",Maiyas,Yes,No,4.0,1778,Jayanagar,Quick Bites,"[coffee, thali, kharabath, dosa, idli, icecrea...","[north indian, south indian, fast food, bevera...",200,"[1.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 5.0, 4.0, ...",sunday time taken provide seating minutes way...,4,"[coffee, thali, kharabath, dosa, idli, icecrea...",3.8,12.9292731,77.5824229,0.000000,0.000000
78,"295/A, 7th Main road, 7th Block, 3rd Stage, Ba...",Mane Thindi,No,No,3.7,130,Banashankari,Quick Bites,"[coffee, tea, kesari bath, dosa, vada, idli, d...",[south indian],200,"[3.0, 4.0, 2.0, 3.0, 3.0, 3.0, 3.0, 4.0, 2.0, ...",ambience might greatest suite dosas next time ...,1,"[coffee, tea, kesari bath, dosa, vada, idli, d...",3.6,12.965283249999999,77.59445195,0.048039,4.210993
